# Evolver Loop 3 Analysis

Investigate the 70.523320 score submission from snapshot 21328309254

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
import json
import os
from shapely.geometry import Polygon
from shapely import affinity
from collections import defaultdict

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

In [2]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx = min(mnx, X)
            mxx = max(mxx, X)
            mny = min(mny, Y)
            mxy = max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

In [3]:
def make_tree_polygon(x, y, deg):
    """Create a Shapely polygon for a tree at given position and rotation"""
    p = Polygon(zip(TX, TY))
    p = affinity.rotate(p, deg, origin=(0,0))
    p = affinity.translate(p, x, y)
    return p

def check_overlaps_strict(xs, ys, degs, buffer_dist=1e-6):
    """Check for overlaps with strict tolerance"""
    n = len(xs)
    if n <= 1:
        return True, []
    
    polygons = [make_tree_polygon(xs[i], ys[i], degs[i]) for i in range(n)]
    
    overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]):
                dist = polygons[i].distance(polygons[j])
                if dist < buffer_dist:
                    intersection = polygons[i].intersection(polygons[j])
                    if intersection.area > 1e-12:
                        overlaps.append((i, j, intersection.area))
    
    return len(overlaps) == 0, overlaps

In [4]:
def parse_submission(filepath):
    """Parse submission CSV and return dict of N -> (xs, ys, degs)"""
    df = pd.read_csv(filepath)
    df['x_val'] = df['x'].str.replace('s', '').astype(float)
    df['y_val'] = df['y'].str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
    df['N'] = df['id'].str.split('_').str[0].astype(int)
    
    configs = {}
    for n, group in df.groupby('N'):
        xs = group['x_val'].values
        ys = group['y_val'].values
        degs = group['deg_val'].values
        configs[n] = (xs, ys, degs)
    
    return configs

def calculate_total_score(configs, tx, ty):
    """Calculate total score across all N values"""
    total = 0.0
    scores_by_n = {}
    for n in range(1, 201):
        if n in configs:
            xs, ys, degs = configs[n]
            score = score_group(xs, ys, degs, tx, ty)
            scores_by_n[n] = score
            total += score
    return total, scores_by_n

In [5]:
# Load the 70.523320 submission
better_path = '/home/nonroot/snapshots/santa-2025/21328309254/code/experiments/003_valid_ensemble/submission.csv'
better_configs = parse_submission(better_path)
better_total, better_scores = calculate_total_score(better_configs, TX, TY)
print(f"Better submission score: {better_total:.6f}")

Better submission score: 70.523320


In [6]:
# Check for overlaps in all N values
print("\nChecking for overlaps in all N values...")
failed_n = []
for n in range(1, 201):
    xs, ys, degs = better_configs[n]
    valid, overlaps = check_overlaps_strict(xs, ys, degs)
    if not valid:
        failed_n.append((n, len(overlaps)))

print(f"\nN values with overlaps: {len(failed_n)}")
for n, num_overlaps in failed_n[:20]:
    print(f"  N={n}: {num_overlaps} overlaps")


Checking for overlaps in all N values...



N values with overlaps: 57
  N=2: 1 overlaps
  N=4: 4 overlaps
  N=5: 10 overlaps
  N=16: 21 overlaps
  N=40: 37 overlaps
  N=46: 39 overlaps
  N=47: 40 overlaps
  N=48: 48 overlaps
  N=53: 86 overlaps
  N=54: 89 overlaps
  N=55: 91 overlaps
  N=56: 93 overlaps
  N=59: 55 overlaps
  N=62: 95 overlaps
  N=69: 62 overlaps
  N=70: 63 overlaps
  N=71: 120 overlaps
  N=77: 101 overlaps
  N=78: 95 overlaps
  N=79: 94 overlaps


In [7]:
# Compare with our current best
current_best_path = '/home/code/submission_candidates/candidate_002.csv'
current_configs = parse_submission(current_best_path)
current_total, current_scores = calculate_total_score(current_configs, TX, TY)
print(f"\nCurrent best score: {current_total:.6f}")
print(f"Better submission score: {better_total:.6f}")
print(f"Improvement: {current_total - better_total:.6f}")


Current best score: 70.615745
Better submission score: 70.523320
Improvement: 0.092425


In [8]:
# If the better submission has overlaps, we need to repair it
# Load baseline for fallback
baseline_path = '/home/code/preoptimized/submission.csv'
baseline_configs = parse_submission(baseline_path)
baseline_total, baseline_scores = calculate_total_score(baseline_configs, TX, TY)
print(f"Baseline score: {baseline_total:.6f}")

Baseline score: 70.647327


In [9]:
# Create a repaired version - use better submission where valid, baseline where overlapping
if failed_n:
    print(f"\nRepairing {len(failed_n)} N values with overlaps...")
    repaired_configs = {}
    for n in range(1, 201):
        if n in [x[0] for x in failed_n]:
            # Use baseline for overlapping N values
            repaired_configs[n] = baseline_configs[n]
        else:
            # Use better submission
            repaired_configs[n] = better_configs[n]
    
    repaired_total, repaired_scores = calculate_total_score(repaired_configs, TX, TY)
    print(f"Repaired submission score: {repaired_total:.6f}")
    print(f"Improvement over current best: {current_total - repaired_total:.6f}")
else:
    print("\nNo overlaps found! The better submission is valid.")
    repaired_configs = better_configs
    repaired_total = better_total


Repairing 57 N values with overlaps...
Repaired submission score: 70.618925
Improvement over current best: -0.003180


In [10]:
# Save the repaired submission
def generate_submission(configs, output_path):
    rows = []
    for n in range(1, 201):
        xs, ys, degs = configs[n]
        for i in range(len(xs)):
            row = {
                'id': f'{n:03d}_{i}',
                'x': f's{xs[i]}',
                'y': f's{ys[i]}',
                'deg': f's{degs[i]}'
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    return df

os.makedirs('/home/submission', exist_ok=True)
submission_df = generate_submission(repaired_configs, '/home/submission/submission.csv')
print(f"\nSaved submission to /home/submission/submission.csv")
print(f"Total rows: {len(submission_df)}")


Saved submission to /home/submission/submission.csv
Total rows: 20100


In [11]:
# Verify the saved submission
verify_configs = parse_submission('/home/submission/submission.csv')
verify_total, verify_scores = calculate_total_score(verify_configs, TX, TY)
print(f"\nVerification of saved submission: {verify_total:.6f}")
print(f"Expected: {repaired_total:.6f}")
print(f"Match: {abs(verify_total - repaired_total) < 1e-6}")


Verification of saved submission: 70.618925
Expected: 70.618925
Match: True


In [ ]:
# Final summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Baseline score: {baseline_total:.6f}")
print(f"Current best (candidate_002): {current_total:.6f}")
print(f"Better submission (70.523320): {better_total:.6f}")
print(f"Repaired submission: {verify_total:.6f}")
print(f"\nImprovement over current best: {current_total - verify_total:.6f}")
print(f"Target: 68.894234")
print(f"Gap to target: {verify_total - 68.894234:.6f}")